# Simulation of quantum systems

In this notebook, we introduce the concepts of quantum simulation and the basic usage of SimuQ. 

Given a Hamiltonian $H$, the time evolution of a quantum system is governed by the Schrödinger equation. $$i\frac{d}{dt}|\psi(t)\rangle = H|\psi(t)\rangle.$$

Simulation of quantum systems is to solve the Schrödinger equation and obtain the system's state at a given time.

Due to the exponentially-large Hilbert space, a classical computer can not effectively simulate a quantum system's time evolution in general. SimuQ provides a more general and flexible way to simulate the time evolution of a quantum system using quantum computers.



## Installation

You may install SimuQ directly via `pip`:

In [ ]:
%pip install simuq

We start with programming the Ising model on a 3 qubit chain.

# Prepare the python environment


In [2]:
from simuq.qsystem import QSystem
from simuq.environment import qubit

Here `QSystem` is the class for quantum systems, and `qubit` is the class for qubit sites.

## Define the evolution

First we create a quantum system and a list of qubit sites.

In [2]:
n_qubits = 3
qs = QSystem()
q = [qubit(qs) for i in range(n_qubits)]

Suppose our target is a short evolution governed by an constant Ising Hamiltonian $H=X_1X_2+X_2X_3+Z_1+Z_2+Z_3$. We can program the evolution as follows.

In [3]:
h = q[0].X * q[1].X + q[1].X * q[2].X + q[0].Z + q[1].Z + q[2].Z

We add a $T=1$ time evolution under $H$ to the quantum system.


In [4]:
T = 1
qs.add_evolution(h, T)

Then `qs` contains the evolution of $H$ for time $T$. To add next segment of evolution, we only need to call the method `add_evolution` again.

# Create a QuTiP provider

A provider is a user interface for convenient manipulations of functionalities of SimuQ. QuTiP is a python package for simulating the dynamics of open quantum systems. We use QuTiP provider as a basic example on how to use providers to deploy quantum simulation problems on devices and obtain results.

We can create a QuTiP provider via the following code


In [6]:
from simuq.providers import QuTiPProvider
qpp = QuTiPProvider()

## Compilation in provider

To simulate a quantum system `qs` programmed in HML via SimuQ, we need three major steps of a provider: `compile`, `run`, `results`.

We call the `compile` function of the provider to process the system into a runnable executable. For QuTiP provider, we can execute

In [7]:
qpp.compile(qs)

Compiled.


QuTiP provider processes the quantum system `qs` and translate it into a Hamiltonian in QuTiP. 

For other providers, compile command may specify the backend device, AAIS, and compiler specifications.

When compilation succeeds, the job will be recorded in the provider.

## Run and obtain results from providers

Running a job will send the compilation results to backend devices to execute. For QuTiP provider, we execute


In [8]:
qpp.run()

Solved.


For other providers, you can choose whether to run on classical simulators, and how many shots to run on the real device or simulators. Note that these classical simulators are mostly from hardware providers and executed on the cloud.

To retrieve the results, we can execute

In [9]:
qpp.results()

{'000': 0.6972053600815633,
 '001': 0.0,
 '010': 0.0,
 '011': 0.059733589164270996,
 '100': 0.0,
 '101': 0.18332746158989463,
 '110': 0.059733589164270996,
 '111': 0.0}

If the task is still in the queue on the device, an exception will be raised. When the retrieval succeeds, a dictionary is returned, which contains the frequencies of obtaining a measurement array (encoded as a 0/1 string). A bit in the string corresponds to a site of the quantum system. 

We can call the following code to show the order of the sites in the measurement output.

In [10]:
qpp.print_sites()

Order of sites: ['Qubit0', 'Qubit1', 'Qubit2']


# Compile and run on devices

## Declare IonQ providers

Accesses to IonQ Quantum Cloud require an API key from IonQ.

Assuming that you have a string `API_key` storing it, we can declare an IonQ provider in SimuQ by

In [5]:
from simuq.providers import IonQProvider
iqp = IonQProvider(from_file="../../../ionq_API_key")

## Compile, run, and retrieve results

To compile and run the 3-site Ising system `qs` on IonQ devices, we need to specify a device and an AAIS. By default, the device is Aria-1 and the AAIS in compilation is Heisenberg-AAIS. One can also pass detailed parameters in the compilation like error tolerance bound and number of Trotterization here. For simplicity, we execute with the default compiler configuration by

In [10]:
iqp.compile(qs)

We then run the experiment on simulator, which executes


In [11]:
iqp.run(shots=4096, on_simulator=True)

{'id': '6abbd333-0786-469e-b26e-f586d40b44f7', 'status': 'ready', 'request': 1693878157}


This will submit a job to IonQ Quantum Cloud using the compiled results, and print the job id and status. The simulator will account for the noise model of Aria-1. To retrieve the results, we wait for a short period of time (normally several seconds) for the cloud server to queue and process the simulation, and then run


In [12]:
iqp.results()

{'000': 0.686469675, '011': 0.062272307, '101': 0.18898571, '110': 0.062272307}

To run on real devices, we can execute



In [ ]:
iqp.run(shots = 4096)

Here `shots` represent the number of repetition of the experiment. After queuing and executing, you may retrieve the results by calling `iqp.results()`.

# Time dependent simulation
SimuQ also support simulating time dependent Hamiltonians. We use QAOA as an example to show how to program a time dependent Hamiltonian.

QAOA, or Quantum Approximate Optimization Algorithm is a class of variational quantum algorithms to solve certain combinatorial optimization problems. The algorithm is based on iteratively apply $U_C=e^{-\gamma_i H_c}$ and $U_B=e^{-\beta_i H_B}$ to the initial state $|+\rangle^{\otimes n}$ to get the final result. 

For the max-cut problem, $H_B=\sum_{(i,j)\in E} Z_i\otimes Z_i$ and $H_C=\sum_i X_i$. We can program the QAOA problem for a 12-cycle using SimuQ as follows.

In [8]:
def GenQS(n=12, p=3):
    qs = QSystem()
    ql = [qubit(qs) for i in range(n)]
    link = [(i, (i + 1) % n) for i in range(n)]
    parameter_list = [
        0.5702193,
        -0.58631086 / 2,
        0.85160685,
        -1.7058538 / 2,
        0.29468536,
        -1.132814 / 2,
    ] # These are optimal parameters calculated offline
    for i in range(p):
        h = 0
        for q0, q1 in link:
            h += parameter_list[2 * i] * ql[q0].Z * ql[q1].Z
        qs.add_evolution(h, 1)
        h = 0
        for q0 in range(n):
            h += parameter_list[2 * i + 1] * ql[q0].X
        qs.add_evolution(h, 1)
    return qs
QAOA=GenQS()

Next, we use IBM's provider to run the simulation.

In [5]:
from simuq.providers import IBMProvider

ibm = IBMProvider(from_file='../../../qiskit_APIKEY',hub="ibm-q-ornl", group="ornl", project="phy147")

We need to specify the initial state, this can be done with `state_prep` parameters.

In [6]:
from qiskit import QuantumCircuit
state_prep=QuantumCircuit(12)
state_prep.h(range(12))

In [11]:
ibm.compile(QAOA, backend='ibmq_guadalupe', trotter_num = 1,use_pulse=False, state_prep = state_prep)
ibm.run(on_simulator = True, shots = 8192)
result=ibm.results(on_simulator = True)

To understant the quality of the result, we calculated the average cut.

In [12]:
def average_cut(result):
    avg_cut=0
    for key in result.keys():
        cut_i=0
        for i in range(12):
            if key[i]!=key[(i+1)%12]:
                cut_i+=1
        avg_cut+=result[key]*cut_i
    return avg_cut
average_cut(result)

10.503662109375

# Non qubit systems

Thanks to the abstraction of a `site`, simuq can handle non-qubit systems as well with a uniform interface.

In [17]:
from simuq.environment import fermion
from simuq.qsystem import QSystem

tau = 3.3
alpha = 3
Jmax = 1
D = 3

qs = QSystem()
f = [fermion(qs) for i in range(3)]

gamma_x = [f[i].a + f[i].c for i in range(3)]
gamma_y = [-1j * (f[i].a - f[i].c) for i in range(3)]


def model(alpha, J01, J12, J20):
    J = [[0, J01, -J20], [-J01, 0, J12], [J20, -J12, 0]]

    h = 0
    for i in range(3):
        h += 1j * alpha * gamma_x[i] * gamma_y[i]
    for i in range(3):
        for j in range(3):
            h += 0.5j * J[i][j] * gamma_x[i] * gamma_x[j]

    return h


def branch(J01_init, J01_end, J12_init, J12_end, J20_init, J20_end):
    def ret(t):
        J01 = J01_init + (J01_end - J01_init) * (t / tau)
        J12 = J12_init + (J12_end - J12_init) * (t / tau)
        J20 = J20_init + (J20_end - J20_init) * (t / tau)
        return model(alpha, J01, J12, J20)

    return ret
qs.add_evolution(model(3, 1, 1, 1), 1)
from simuq.transformation import JW_transform
new_qs, new_sites = JW_transform(qs)


In [19]:
new_qs.evos[0][0].ham

[(['Z', '', ''], (-3+0j)),
 (['', 'Z', ''], (-3+0j)),
 (['', '', 'Z'], (-3+0j)),
 (['Y', 'X', ''], (1-0j)),
 (['Y', 'Z', 'X'], (-1+0j)),
 (['', 'Y', 'X'], (1-0j))]